In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys,os,glob
import subprocess
sys.path.insert(0,'../src')

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm,Normalize,SymLogNorm

# data format:

* `TIGRESS` simulations dump binary data files with a header following a specific format called `VTK`.
* `VTK` file contains main physical quantities, e.g., `density`, `pressure`, `velocity`, `magnetic field`, etc.
* (If post-processed) UV radiation fields (FUV and EUV) are then calculated by radiation transfer using adaptive ray tracing and stored in a separate `VTK` file. 

# exmaple data:

* example data files can be downloaded at [this URL](http://tigress-web.princeton.edu/~changgoo/TIGRESS_example_data/)

In [22]:
# TIGRESS VTK dump reader and units
from vtk_reader import *
import set_units

# mean molecular weight per hydrogoen (adopted in the simulation)
muH=1.4271
# unit setup
units=set_units.set_units(muH=muH)

# print unit system (utilizing astropy.constants and units)
for k,v in units.items():
    print(k,v)

muH 2.3869987106358e-24 g
density 0.035268224298483024 solMass / pc3
velocity 1.0 km / s
length 1.0 pc
time 0.9777922216731284 Myr
pressure 2.3869987106358004e-14 erg / cm3
gravitational_potential 1.0 km2 / s2
number_density 1.0 1 / cm3
magnetic_field 0.5476852239548456 uG
temperature 1.0 K
mass 0.03526822429848302 solMass
Gcode 2.9365549587313344e-11 cm6 / (g2 s4)


### NOTE: conversion between variables and unit conversion should be trivial except `temperature`, which involves a tabulated mean molecular weight.

In [25]:
data_folder='/tigress/changgoo/public_html/TIGRESS_example_data'

In [26]:
ds=AthenaDataSet('{}/MHD_4pc_new_joined.0300.vtk'.format(data_folder))

# print data domain information
for k,v in ds.domain.items():
    if k != 'field_map':
        print(k,v)

left_edge [ -512.  -512. -3584.]
right_edge [ 512.  512. 3584.]
dx [4. 4. 4.]
Lx [1024. 1024. 7168.]
center [0. 0. 0.]
Nx [ 256  256 1792]
ndim 3
time 300.0009
nscal 0


In [27]:
# read hydrogen number density [returns 3D array]
nH=ds.read_all_data('density')
print(nH.shape)

(1792, 256, 256)


In [28]:
# available fields
print(ds.field_list)

['density', 'velocity', 'pressure', 'gravitational_potential', 'magnetic_field']


In [ ]:
# velocity and magnetic field are vector [returns 4D array]
Bvec=ds.read_all_data('magnetic_field')
# convert to microgauss
Bvec *= units['magnetic_field'].value
print(Bvec.shape)

In [ ]:
Bx=Bvec[...,0]
By=Bvec[...,1]
Bz=Bvec[...,2]

In [ ]:
dx,dy,dz=ds.domain['dx']

In [ ]:
# calculate surface density
surf = (nH*dz).sum(axis=0)
# unit conversion to (M_sun/pc^2)
surf = (surf*units['density']*units['length']).to('M_sun/pc^2')
# to hydrogen column density
import astropy.constants as ac
NH = (surf/(muH*ac.m_p)).to('cm^(-2)')

In [ ]:
print(surf.mean(),NH.mean())

In [ ]:
# example visualization
le=ds.domain['left_edge']
re=ds.domain['right_edge']

fig,axes=plt.subplots(1,3,figsize=(15,6))

extent=[le[0],re[0],le[1],re[1]]
Nz, Ny, Nx = nH.shape
hNz = int(Nz/2)
nmid = nH[hNz,:,:] # midplane slice
n500 = nH[hNz+int(500/dz),:,:] # z=500pc

for ax, slc in zip(axes,[surf.value, nmid, n500]):
    im = ax.imshow(slc,norm=LogNorm(),extent=extent)
    cbar = plt.colorbar(im,ax=ax,orientation='horizontal')
    ax.set_xlabel('x [pc]')
    ax.set_ylabel('y [pc]')

axes[0].set_title(r'surface density')
axes[1].set_title(r'number density at z=0pc')
axes[2].set_title(r'number density at z=500pc')

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(12,7))

x = np.arange(le[0],re[0],dx) + 0.5*dx # cell-center positions
y = np.arange(le[1],re[1],dy) + 0.5*dy # cell-center positions

kidx=[hNz, hNz+int(500/dz)]
for ax,k0 in zip(axes,kidx):
    nslc=nH[k0,:,:]
    Bxslc=Bx[k0,:,:]
    Byslc=By[k0,:,:]
    Bmag=np.sqrt(Bxslc**2+Byslc**2)
    im=ax.imshow(nslc,cmap=plt.cm.binary,norm=LogNorm(vmin=1.e-4,vmax=100),extent=extent)
    st=ax.streamplot(x,y,Bxslc,Byslc,
                     color=Bmag,cmap=plt.cm.plasma,norm=LogNorm(vmin=1.e-2,vmax=10))
    ax.set_xlim(le[0],re[0])
    ax.set_ylim(le[1],re[1])
    ax.set_xlabel('x [pc]')
    ax.set_ylabel('y [pc]')
axes[0].set_title(r'slice at z=0pc')
axes[1].set_title(r'slice at z=500pc')
plt.colorbar(im,ax=axes[0],orientation='horizontal',label=r'number density $[{\rm cm}^{-3}]$')
plt.colorbar(st.lines,ax=axes[1],orientation='horizontal',label=r'magnetic field strength $[\mu{\rm G}]$')